### Tutorial - Introduction to scRNA-seq analysis using Scanpy
This Tutorial was guided heavily by Malte Luecken's "Best Practices" Notebook. (PostDoc from TheisLab) <br>
The original notebook can be found at https://github.com/theislab/single-cell-tutorial/blob/master/latest_notebook/Case-study_Mouse-intestinal-epithelium_1906.ipynb

It is highly recommended to look at the Scanpy Documentation to see a more detailed explanation on the function and other parameters one can specifiy: <br>
https://scanpy.readthedocs.io/en/stable/  

### Webtool to Longitudinal single cell transcriptomics:
https://theislab.github.io/LungInjuryRegeneration/
### Webtool to IPF Cell Atlas:
http://www.ipfcellatlas.com/

In [ ]:
## Import Libraries used in this script
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
import seaborn as sb

import warnings
warnings.filterwarnings("ignore")

## Define a nice colour map for gene expression
from matplotlib import colors
gray_red = colors.LinearSegmentedColormap.from_list("", ["lightgray", "red", "darkred"], N = 128)

sc.settings.figdir = "Plots/"
sc.set_figure_params(vector_friendly = True)
plt.rcParams.update({"font.size": 14})
sb.set_style("ticks")

sc.logging.print_version_and_date()

### Read in Data File
The raw count matrix is stored as a .h5ad file, which can be read in directly using Scanpy’s read() function

In [ ]:
adata = ## read it in ##
adata

In [ ]:
### Show the first 10 gene names (stored in adata.var_names) ###
print(<TO_INSERT>)

### Show the first 10 cell ids (stored in adata.obs_names) ###
print(<TO_INSERT>)

## Show the expression values of the first 10 genes in the first 5 cells
## adata objects store the expression matrix in .X layer (cells (rows) x genes (columns))
adata.X[0:5, 0:10].todense()

In [ ]:
## Show how many cells you have per age group, which is stored in adata.obs.grouping
## Use the function .value_counts()

### Preprocessing and Quality Control
Data quality control can be split into cell QC and gene QC. Typical quality measures for assessing the quality of a cell include the number of molecule counts (UMIs), the number of expressed genes, and the fraction of counts that are mitochondrial. A high fraction of mitochondrial reads being picked up can indicate cell stress, as there is a low proportion of nuclear mRNA in the cell. It should be noted that high mitochondrial RNA fractions can also be biological signals indicating elevated respiration.

In [ ]:
# Quality control - plot QC Metrics
plt.rcParams['figure.figsize'] = (8, 6)

## use scanpy’s violin plot to visualize the quality of the cells per "identifier" ##

In [ ]:
#Data quality summary plots
sc.pl.scatter(adata, "n_counts", "n_genes", color = "identifier")

## Zoom in to range
sc.pl.scatter(adata[adata.obs["n_counts"] < 8000], "n_counts", "n_genes", color = "percent_mito")

We can assess whether there are cells with unexpected summary statistics. For example, there is a cloud of points with many counts, but few genes. This cloud of cells may indicate empty droplets containing ambient RNA, or simply dying cells.

Cells with lower counts and genes tend to have a higher fraction of mitochondrial counts. These cells are likely under stress or are dying. When apoptotic cells are sequenced, there is less mRNA to be captured in the nucleus, and therefore fewer counts overall, and thus a higher fraction of counts fall upon mitochondrial RNA. If cells with high mitochondrial activity were found at higher counts/genes per cell, this would indicate biologically relevant mitochondrial activity.

In [ ]:
#Thresholding decision: counts
sb.distplot(adata.obs['n_counts'], kde = False)
plt.show()

## Zoom into lower end of the distribution
sb.distplot(adata.obs['n_counts'][adata.obs['n_counts'] < 5000], kde = False, bins=60)
plt.axvline(<TO_INSERT>, color = "blue")
plt.show()

In [ ]:
#Thresholding decision: genes
sb.distplot(adata.obs['n_genes'], kde=False, bins=60)
plt.show()

## Zoom into lower end of the distribution
sb.distplot(adata.obs['n_genes'][adata.obs['n_genes'] < 3000], kde = False, bins = 60)
plt.axvline(<TO_INSERT>, color = "blue")
plt.show()

In [ ]:
## Actually do the Filtering
print('Total number of cells: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, max_counts = <TO_INSERT>)
print('Number of cells after max count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_counts = <TO_INSERT>)
sc.pp.filter_cells(adata, min_genes = <TO_INSERT>)
print('Number of cells after min count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_genes(adata, min_cells = 3)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))

adata = adata[adata.obs["percent.mito"] < 0.1]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

The filtering is performed based on the thresholds we identified from the QC plots. Genes are also filtered if they are not detected in at least 3 cells. This reduces the dimensions of the matrix by removing 0 count genes and genes which are not sufficiently informative of the dataset. <br>
In general it is a good idea to be permissive in the early filtering steps, and then come back to filter out more stringently when a clear picture is available of what would be filtered out. 

### Quality Plots after Filtering

In [ ]:
### Re-evaluate your quality using similar plotting methods as above ### 

In [ ]:
## Summary Statistics after Filtering
info = pd.DataFrame(data = adata.obs["identifier"].cat.categories, columns = ["identifier"])
info["n_counts"] = adata.obs.groupby(["identifier"])["n_counts"].median().values
info["n_genes"] = adata.obs.groupby(["identifier"])["n_genes"].median().values
info["percent_mito"] = adata.obs.groupby(["identifier"])["percent_mito"].mean().values
info["n_cells"] = adata.obs.groupby(["identifier"])["n_genes"].size().values
info

### Normalization and log Transformation
Up to this point the data is only available as a count matrix. Counts are representative of molecules that were captured in the scRNA-seq experiment. As not all mRNA molecules in a cell are captured, there is a variability in the total number of counts detected between cells that results from both the number of molecules that were in the cells, and the sampling. As we cannot assume that all cells contain an equal number of molecules (cell sizes can differ substantially), we have to estimate the number of molecules that were initially in the cells. We keep this data in the '.X' part of the AnnData object as it will be used to visualize gene expression and perform statistical tests such as computing marker genes for clusters.

In [ ]:
#Keep the count data in a counts layer 
adata.layers["counts"] = adata.X.copy()

In [ ]:
adata.layers["counts"][0:5, 0:10].todense()

In [ ]:
## Normalization (divide by total UMI counts per cell)
### look at scanpy’s preprocessing methods for normalization ### 

In [ ]:
## Log Transformation
### look at scanpy’s preprocessing methods for Log Transformation ### 

In [ ]:
## show number of cells x number of genes
adata.shape

### Extract highly variable Genes

We extract highly variable genes (HVGs) to further reduce the dimensionality of the dataset and include only the most informative genes. Genes that vary substantially across the dataset are informative of the underlying biological variation in the data. As we only want to capture biological variation in these genes, we select highly variable genes after normalization and batch correction. HVGs are used for clustering, trajectory inference, and dimensionality reduction/visualization, while the full data set is used for computing marker genes, differential testing and visualizing expression values on the data.

In [ ]:
sc.pp.highly_variable_genes(adata, flavor = "seurat", n_top_genes = 4000, batch_key = "identifier")
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))

In [ ]:
### Find a visualization to show characeristics of the chosen genes ###

Highly variable gene information is stored automatically in the adata.var['highligh_variable'] field. The dataset now contains:

- log-normalized data in adata.X
- highly variable gene annotations in adata.var['highly_variable']

### Dimensionality Reduction using Principal Component Analysis
Visualizing scRNA-seq data is the process of projecting a high-dimensional matrix of cells and genes into a few coordinates such that every cell is meaningfully represented in a two-dimensional graph. Tt is a good idea to look at several visualizations and decide which visualization best represents the aspect of the data that is being investigated. <br>
Overall t-SNE visualizations have been very popular in the community, however the recent UMAP algorithm has been shown to better represent the topology of the data.


In [ ]:
### Perform a Principal Component Analysis using scanpy’s preprocessing methods ###
### Remember to specify to use only the previously selected highly variable genes for this step ###

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs = 40)

In [ ]:
### Calculate nearest neighbourhood graph and find fitting values for the parameter n_neighbors, n_pcs ###

In [ ]:
### Generate a 2D embedding using UMAP and plot the result ###
plt.rcParams['figure.figsize'] = (6, 5)

In [ ]:
## You can see the annotation of each cell (can be used to colour the cells) as the columns of
adata.obs.head()

### Save the Plot

In [ ]:
### Find out how to save the plot as PDF. Hint: look at the save parameter of sc.pl.umap()

### Clustering
Clustering is a central component of the scRNA-seq analysis pipeline. To understand the data, we must identify cell types and states present. The first step of doing so is clustering. This modularity optimization is performed on the k-nearest-neighbour graph of cells in scRNA-seq analysis.  <br>
By changing the resolution parameter one can influence the number of clusters the algorithm finds. Investigating several resolutions allows us to select a clustering that appears to capture the main clusters in the visualization and can provide a good baseline for further subclustering of the data to identify more specific substructures.

In [ ]:
### Generate an unsupervised Clustering using scanpy’s tl.leiden() function ###

In [ ]:
## You can increase the plot by setting following Paramter in the funtion
ax = plt.figure(figsize = (5, 4), dpi = 120).gca()

## One can directly plot the labels on top of the cells by setting
legend_loc = "on data"

One can also visualize expression of certain known marker genes by specifying the colour Parameter

In [ ]:
sc.pl.umap(adata, color = ["Ear2", "Sftpd", "Foxj1"], cmap = gray_red, size = 20)

### Find Marker Genes and Annotate Clusters
To annotate the clusters we obtained, we find genes that are up-regulated in the cluster compared to all other clusters (marker genes). The test is automatically performed on the .raw data set, which is uncorrected and contains all genes. All genes are taken into account, as any gene may be an informative marker.

In [ ]:
### Generate a markers table using scanpy’s rank_genes_groups() and specify the group info ###

In [ ]:
## Plot the first 5 genes per cluster
sc.pl.rank_genes_groups_dotplot(adata, n_genes = 5)

To further identify the clusters in ones data set, one can look at the overlap with a list of known marker genes.
In practice marker gene sets can be obtained from public databases such as Linnarson's mouse brain atlas, various Human Cell Atlas datasets, and other published reference atlases. It should be noted that marker genes may not always overlap as expected given that atlases tend to be investigated under wild-type conditions.

### Pick one or two cluster for which you could identify the cell type and annotate them in object
As an example, here we are showing the expression of 3 well known marker genes for cell types of the lung.  
Feel free to use other marker genes that you have in mind, or to check out the marker genes saved in Plot/dotplot_cell_type_marker.pdf

In [ ]:
sc.pl.violin(adata, keys = ["Sftpd"], groupby = "leiden_1", size = 1.5)    ## AT2 cell marker
sc.pl.violin(adata, keys = ["Foxj1"], groupby = "leiden_1", size = 1.5)    ## Ciliated cell marker
sc.pl.violin(adata, keys = ["Ear2"], groupby = "leiden_1", size = 1.5)     ## Macrophage marker

In [ ]:
sc.pl.dotplot(adata, var_names = ["Ear2", "Sftpd", "Foxj1"], groupby = "leiden_1", dendrogram = True)

In [ ]:
## With this code snippet, you can highlight certain cluster of interest in the UMAP
sc.pl.umap(adata, color = "leiden_1", legend_loc = "on data", 
           groups = ["1", "17", "8", "14", "20", "6", "0", "10", "4"])

In [ ]:
## You can pick certain cluster and assign them a cell type as following
celltype_map = {"1": "Macrophages", "4": "Ciliated cells", "17": "AT2 cells", "8": "AT2 cells",
                "14": "AT2 cells", "20": "AT2 cells", "6": "AT2 cells", "0": "AT2 cells", "10": "AT2 cells",
                "17": "Doublets"}

adata.obs["cell_type"] = [celltype_map[leiden] if leiden in celltype_map.keys() else leiden
                          for leiden in adata.obs.leiden]

In [ ]:
sc.pl.umap(adata, color = "cell_type", legend_loc = "on data", ax = plt.figure(figsize=(6, 5), dpi = 100).gca())

### Pick one Example Cluster for Differential Gene Expression

In [ ]:
## Create a new object, using only cells assigned to
cell_type = <TO_INSERT>
sub = adata[adata.obs["cell_type"] == cell_type].copy()
sc.pl.umap(sub, color = ["cell_type", "grouping"])

After subsetting the data, it makes sense to re-caculate the PCs and knn graph, as the neighbourhood of the individual cells has likely changed. This also helps enhance certain features which were over-shadowed in the full embedding

In [ ]:
### recalculate PCA, k nearest neighbor graph and the UMAP ###
### Keep in mind to use the right object in the functions :) ###

### Perform Differential Gene Expression

In [ ]:
### Use tl.rank_genes_groups to find genes upregulated in either young or old mice ###
## Examine the results e.g. by sc.pl.rank_genes_groups_dotplot ###

### Potential Bonus: PAGA
By quantifying the connectivity of partitions (groups, clusters) of the single-cell graph, partition-based graph abstraction (PAGA) generates a much simpler abstracted graph of partitions, in which edge weights represent confidence in the presence of connections. By tresholding this confidence in paga(), a much simpler representation of the manifold data is obtained.

In [ ]:
### Perform PAGA on the whole object ###

### You can save the object and all annotations calculated so far with the following command

In [ ]:
adata.write("Data/Aging_processed.h5ad")